In [96]:
import geopandas as gpd
import pandas as pd
import re
import fiona
fiona.Env(); # needed to resolve driver error when exporting to GeoJSON
import numpy as np

### Read in the SLE GeoJSON for 2015 to 2018

In [97]:
sle_data = gpd.read_file('./data/sle/output/sle2015_2018.geojson')

In [98]:
sle_data.head()

,date,city,region,positive,title,geometry
0,09/14/2015,Mecca,"Riverside County (California, USA)",SLEV,YTD Positive Sentinel Chicken,POINT (-116.0302618249568 33.5276819108859)
1,09/10/2015,Mecca,"Riverside County (California, USA)",SLEV,YTD Positive Sentinel Chicken,POINT (-116.0749179098212 33.54414473222415)
2,09/28/2015,Mecca,"Riverside County (California, USA)",SLEV,YTD Positive Sentinel Chicken,POINT (-116.0788138626815 33.54292818890128)
3,09/23/2015,Mecca,"Riverside County (California, USA)",SLEV,YTD Positive Sentinel Chicken,POINT (-116.0293803087907 33.5347901760566)
4,09/24/2015,Mecca,"Riverside County (California, USA)",SLEV,YTD Positive Sentinel Chicken,POINT (-116.0755536165777 33.54329531806545)


### Make sure all cities are properly capitalized

In [99]:
sle_data['city'] = sle_data['city'].str.title()

### Loop through the regions and remove '(California, USA)'

In [100]:
# Create an empty array that will hold region values
regions = []

# Loop over the region column
for value in sle_data['region']:
    if value is None:
        regions.append('None')
    else:
        # Split the name of the value in pieces by ' C' portion of of the current region name 
        chunks = value.split(' C')
        # Keep the first chunk of the split and append
        county = chunks[0]
        regions.append(county)

### Drop the current region column and re-add with the values from the regions list

In [101]:
# Drop the column
sle_data.drop('region', axis=1, inplace=True)

# Add back in using the region array
sle_data['region'] = regions

### Change values in the title column

In [102]:
# Create an empty array that will hold our title values
titles = []

# Loop over the title column
for title in sle_data['title']:
    if title == 'YTD Positive Sentinel Chicken':
        titles.append('Sentinel Chicken')
    elif title == 'YTD Positive Mosquito Pool':
        titles.append('Mosquito Pool')

### Drop the current title column and re-add with the values from the titles list

In [103]:
# Drop the column
sle_data.drop('title', axis=1, inplace=True)

# Add back in using the region array
sle_data['title'] = titles

### Rearrange columns newly added columns match the previous arrangement

In [104]:
# Add our column names to an array
sle_cols = sle_data.columns.tolist()

# Rearrange our columns
sle_cols_rearranged = ['date','city', 'region', 'positive', 'title', 'geometry']

# Assign new arrangement back to dataframe
sle_data = sle_data[sle_cols_rearranged]

In [105]:
sle_data.head()

,date,city,region,positive,title,geometry
0,09/14/2015,Mecca,Riverside,SLEV,Sentinel Chicken,POINT (-116.0302618249568 33.5276819108859)
1,09/10/2015,Mecca,Riverside,SLEV,Sentinel Chicken,POINT (-116.0749179098212 33.54414473222415)
2,09/28/2015,Mecca,Riverside,SLEV,Sentinel Chicken,POINT (-116.0788138626815 33.54292818890128)
3,09/23/2015,Mecca,Riverside,SLEV,Sentinel Chicken,POINT (-116.0293803087907 33.5347901760566)
4,09/24/2015,Mecca,Riverside,SLEV,Sentinel Chicken,POINT (-116.0755536165777 33.54329531806545)


### Rename columns

In [106]:
sle_data.rename(columns={'region' : 'county', 'positive' : 'virus', 'title' : 'spectype'}, inplace=True)

In [107]:
sle_data.head()

,date,city,county,virus,spectype,geometry
0,09/14/2015,Mecca,Riverside,SLEV,Sentinel Chicken,POINT (-116.0302618249568 33.5276819108859)
1,09/10/2015,Mecca,Riverside,SLEV,Sentinel Chicken,POINT (-116.0749179098212 33.54414473222415)
2,09/28/2015,Mecca,Riverside,SLEV,Sentinel Chicken,POINT (-116.0788138626815 33.54292818890128)
3,09/23/2015,Mecca,Riverside,SLEV,Sentinel Chicken,POINT (-116.0293803087907 33.5347901760566)
4,09/24/2015,Mecca,Riverside,SLEV,Sentinel Chicken,POINT (-116.0755536165777 33.54329531806545)


### There are quite a few null values for city so wil break things out into two separate dataframes and recombine once those get values

In [108]:
sle_fully_populated = sle_data.loc[sle_data['city'].notnull()]
sle_null_city = sle_data.loc[sle_data['city'].isnull()]

### Need lat/lon coordinates in our *sle_null_city* dataframe to be able to reverse geocode, so create these columns from the geometry column

In [109]:
sle_null_city['lon'] = sle_null_city['geometry'].x
sle_null_city['lat'] = sle_null_city['geometry'].y

C:\Users\MBacinsk\AppData\Local\Continuum\anaconda3\envs\mappingEnv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\MBacinsk\AppData\Local\Continuum\anaconda3\envs\mappingEnv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Use `geopy` to reverse geocode based on our lat/lon coordinates.  The MapBox provider will be used, but there a quite a few to choose from

In [110]:
from geopy.geocoders import MapBox
geolocator = MapBox('pk.eyJ1IjoiYmF6aW5pNjI3IiwiYSI6ImVmZTNiM2VlZTlhMjk0NzE3MjU5YmEzZWVkYjUwNjAzIn0.idDK-FJKJn3Q7StDtBOX-g',user_agent="sle-geocode")

# Create an address column to populate the results of the reverse gecoding with
sle_null_city['address'] = sle_null_city.apply(
    lambda row: geolocator.reverse((row['lat'], row['lon'])), axis=1)

C:\Users\MBacinsk\AppData\Local\Continuum\anaconda3\envs\mappingEnv\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [111]:
sle_null_city.head()

,date,city,county,virus,spectype,geometry,lon,lat,address
7,05/16/2016,None,Yuma,SLEV,Sentinel Chicken,POINT (-114.6628399620399 32.62450630721889),-114.662840,32.624506,"(3703 West County 14th Street, Somerton, Arizo..."
8,05/27/2016,None,Yuma,SLEV,Sentinel Chicken,POINT (-114.6673291684298 32.62595933836609),-114.667329,32.625959,"(South Avenue C, Yuma, Arizona 85365, United S..."
13,07/11/2016,None,Yuma,SLEV,Sentinel Chicken,POINT (-114.6637011956284 32.62001534603975),-114.663701,32.620015,"(3762 West Sun Street, Yuma, Arizona 85365, Un..."
18,07/25/2016,None,Yuma,SLEV,Sentinel Chicken,POINT (-114.6681104142361 32.62598977044367),-114.668110,32.625990,"(4115 West County 14th Street, Yuma, Arizona 8..."
97,06/28/2016,None,Riverside,SLEV,Mosquito Pool,POINT (-116.0737912126328 33.52318183677199),-116.073791,33.523182,"(72120 Lincoln Ave, Mecca, California 92254, U..."


### Looks like our first four entries are actually in Arizona so we can drop those based on the county values.  No other rows have a county value of Yuma and are in CA

In [112]:
sle_null_city_no_az = sle_null_city.loc[sle_null_city['county'] != 'Yuma']

### Now we can loop through the address column to pull out the city from the returned address.
### Shapely objects will be converted to strings, otherwise they can't be iterated over to pull out the city name

In [113]:
# Empty list to hold extracted city
cities = []

# Address column to a list 
addies = sle_null_city_no_az['address'].tolist()

# Loop through the addresses, pull out the city and append to cities list 
for addy in addies:
    
    # Convert Shapely object to a string
    addy_as_string = str(addy)
   
    # Regex looking for something like City,California anywhere in our address string
    city_state = re.search("([A-Z][a-z]+)+,\s[C][a]+", addy_as_string)
        
    # If we have a match append the city to our cities list 
    if city_state:
        cities.append(city_state.group(1))

print(cities)

['Mecca', 'Mecca', 'Fresno', 'Lemoore', 'Kerman', 'Fresno', 'Firebaugh', 'Firebaugh', 'Mendota', 'Mendota', 'Firebaugh', 'Fresno', 'Coalinga', 'Firebaugh', 'Firebaugh', 'Firebaugh', 'Firebaugh', 'Firebaugh', 'Firebaugh', 'Firebaugh', 'Firebaugh', 'Firebaugh', 'Fresno', 'Visalia', 'Fresno', 'Fresno', 'Fresno', 'Visalia', 'Fresno', 'Fresno', 'Visalia', 'Visalia', 'Lemoore', 'Lemoore', 'Lemoore', 'Lemoore', 'Lemoore', 'Madera', 'Fresno']


### The null *city* column in our *sle_null_city_no_az* dataframe can now be replaced with our cities list

In [114]:
# Drop our original city column
sle_null_city_no_az.drop(columns=['city'], axis=1, inplace=True)

C:\Users\MBacinsk\AppData\Local\Continuum\anaconda3\envs\mappingEnv\lib\site-packages\pandas\core\frame.py:3930: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [115]:
# Add the column back in, populated with the cities list 
sle_null_city_no_az['city'] = cities

C:\Users\MBacinsk\AppData\Local\Continuum\anaconda3\envs\mappingEnv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### We can drop our lat, lon, and address columns now

In [116]:
sle_null_city_no_az.drop(columns=['lon', 'lat', 'address'], axis=1, inplace=True)

In [117]:
sle_null_city_no_az.head()

,date,county,virus,spectype,geometry,city
97,06/28/2016,Riverside,SLEV,Mosquito Pool,POINT (-116.0737912126328 33.52318183677199),Mecca
108,07/12/2016,Riverside,SLEV,Mosquito Pool,POINT (-116.0702315447287 33.52206039207996),Mecca
257,06/09/2017,Fresno,SLEV,Mosquito Pool,POINT (-119.8905351402999 36.84060898276345),Fresno
269,07/18/2017,Kings,SLEV,Mosquito Pool,POINT (-119.8417328833465 36.38332703027084),Lemoore
281,08/03/2017,Fresno,SLEV,Mosquito Pool,POINT (-120.3300749617084 36.71340983546221),Kerman


### Now we can combine our dateframes.  We can just use concat since they have the same columns

In [118]:
sle_merged = pd.concat([sle_fully_populated,sle_null_city_no_az], sort=True)

In [119]:
sle_merged.head()

,city,county,date,geometry,spectype,virus
0,Mecca,Riverside,09/14/2015,POINT (-116.0302618249568 33.5276819108859),Sentinel Chicken,SLEV
1,Mecca,Riverside,09/10/2015,POINT (-116.0749179098212 33.54414473222415),Sentinel Chicken,SLEV
2,Mecca,Riverside,09/28/2015,POINT (-116.0788138626815 33.54292818890128),Sentinel Chicken,SLEV
3,Mecca,Riverside,09/23/2015,POINT (-116.0293803087907 33.5347901760566),Sentinel Chicken,SLEV
4,Mecca,Riverside,09/24/2015,POINT (-116.0755536165777 33.54329531806545),Sentinel Chicken,SLEV


### Output to geojson

In [120]:
sle_merged.to_file('./data/sle/output/sle2015_2018_cleaned.geojson', driver="GeoJSON")

C:\Users\MBacinsk\AppData\Local\Continuum\anaconda3\envs\mappingEnv\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


### Checked the output with [geojson.io](geojson.io) and noticed that a few points are in Arizona, so those will have to be dropped

In [121]:
# All the points are in Yuma county
sle_merged.loc[sle_merged['county'] == 'Yuma']

,city,county,date,geometry,spectype,virus
9,Yuma,Yuma,06/28/2016,POINT (-114.6692054769902 32.71421646906434),Sentinel Chicken,SLEV
10,Gila Valley,Yuma,06/28/2016,POINT (-114.5544002242594 32.7015640995177),Sentinel Chicken,SLEV
12,Yuma,Yuma,07/11/2016,POINT (-114.6733321336425 32.715117333355),Sentinel Chicken,SLEV
14,Gila Valley,Yuma,07/11/2016,POINT (-114.5500644238585 32.70499937955419),Sentinel Chicken,SLEV
17,Yuma,Yuma,07/25/2016,POINT (-114.6702023915953 32.71511557577822),Sentinel Chicken,SLEV
19,Gila Valley,Yuma,07/25/2016,POINT (-114.5529591370368 32.70235551859694),Sentinel Chicken,SLEV
20,Yuma,Yuma,08/06/2016,POINT (-114.6688277204835 32.7188197513739),Sentinel Chicken,SLEV
21,Gila Valley,Yuma,08/06/2016,POINT (-114.5509169943425 32.70153210108912),Sentinel Chicken,SLEV
22,Yuma,Yuma,08/22/2016,POINT (-114.6677785973156 32.71252704576695),Sentinel Chicken,SLEV
23,Gila Valley,Yuma,08/22/2016,POINT (-114.5488683215499 32.70624469069669),Sentinel Chicken,SLEV


### Drop the rows and make sure only 12 rows are removed

In [122]:
sle_merged.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 819 entries, 0 to 778
Data columns (total 6 columns):
city        819 non-null object
county      819 non-null object
date        819 non-null object
geometry    819 non-null object
spectype    819 non-null object
virus       819 non-null object
dtypes: object(6)
memory usage: 44.8+ KB


In [123]:
sle_merged_no_az = sle_merged.loc[sle_merged['county'] != 'Yuma']

In [124]:
sle_merged_no_az.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 807 entries, 0 to 778
Data columns (total 6 columns):
city        807 non-null object
county      807 non-null object
date        807 non-null object
geometry    807 non-null object
spectype    807 non-null object
virus       807 non-null object
dtypes: object(6)
memory usage: 44.1+ KB


### Convert date column to a pandas date type which will then be converted to a unix timestamp

In [125]:
sle_merged_no_az['date'] = pd.to_datetime(sle_merged_no_az['date'])

C:\Users\MBacinsk\AppData\Local\Continuum\anaconda3\envs\mappingEnv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [126]:
sle_merged_no_az.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 807 entries, 0 to 778
Data columns (total 6 columns):
city        807 non-null object
county      807 non-null object
date        807 non-null datetime64[ns]
geometry    807 non-null object
spectype    807 non-null object
virus       807 non-null object
dtypes: datetime64[ns](1), object(5)
memory usage: 44.1+ KB


### Convert pandas timestamps to unix timestamps

In [127]:
sle_merged_no_az['date'] = sle_merged_no_az['date'].astype(np.int64)

C:\Users\MBacinsk\AppData\Local\Continuum\anaconda3\envs\mappingEnv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [128]:
sle_merged_no_az.head()

,city,county,date,geometry,spectype,virus
0,Mecca,Riverside,1442188800000000000,POINT (-116.0302618249568 33.5276819108859),Sentinel Chicken,SLEV
1,Mecca,Riverside,1441843200000000000,POINT (-116.0749179098212 33.54414473222415),Sentinel Chicken,SLEV
2,Mecca,Riverside,1443398400000000000,POINT (-116.0788138626815 33.54292818890128),Sentinel Chicken,SLEV
3,Mecca,Riverside,1442966400000000000,POINT (-116.0293803087907 33.5347901760566),Sentinel Chicken,SLEV
4,Mecca,Riverside,1443052800000000000,POINT (-116.0755536165777 33.54329531806545),Sentinel Chicken,SLEV


In [129]:
sle_merged_no_az.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 807 entries, 0 to 778
Data columns (total 6 columns):
city        807 non-null object
county      807 non-null object
date        807 non-null int64
geometry    807 non-null object
spectype    807 non-null object
virus       807 non-null object
dtypes: int64(1), object(5)
memory usage: 44.1+ KB


### Test converting back to make sure process works okay

In [130]:
print (pd.to_datetime(sle_merged_no_az['date'], unit='ns'))

0     2015-09-14
1     2015-09-10
2     2015-09-28
3     2015-09-23
4     2015-09-24
5     2015-10-13
6     2015-11-09
11    2016-07-12
15    2016-07-21
16    2016-07-27
24    2016-09-01
27    2016-09-14
28    2016-10-05
29    2017-07-14
30    2017-07-18
31    2017-07-25
32    2017-07-26
33    2017-08-02
34    2017-08-16
35    2017-09-13
36    2017-09-27
37    2018-08-17
38    2018-08-30
39    2015-07-28
40    2015-07-28
41    2015-07-28
42    2015-07-28
43    2015-08-04
44    2015-08-04
45    2015-08-11
         ...    
311   2017-08-24
313   2017-08-25
315   2017-08-25
328   2017-08-31
345   2017-09-06
346   2017-09-06
347   2017-09-06
366   2017-09-07
373   2017-09-08
374   2017-09-08
375   2017-09-08
391   2017-09-13
392   2017-09-13
445   2018-06-13
524   2018-07-17
525   2018-07-17
532   2018-07-18
533   2018-07-18
551   2018-07-24
564   2018-07-24
608   2018-08-07
661   2018-08-16
662   2018-08-16
686   2018-08-20
704   2018-08-27
705   2018-08-27
706   2018-08-27
707   2018-08-

### Conversion back seems to have worked okay so can output to GeoJSON now

### Only 12 rows were dropped so we can output to GeoJSON again

In [131]:
sle_merged_no_az.to_file('./data/sle/output/sle2015_2018_cleaned.geojson', driver="GeoJSON")